In [ ]:
from dask.distributed import Client, LocalCluster
import bokeh
import geopandas as gpd
import scipy
import libpysal
import momepy
import json
from dask import delayed
import glob
import dask
import dask.dataframe as dd
import momepy
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import json
from tqdm import tqdm
from shapely.geometry import Point
import networkx as nx
import pickle
import time

In [ ]:
local_crs = 3414

place = "singapore"

latlng = (103.85, 1.29)

In [ ]:
daskCluster = LocalCluster(threads_per_worker=1,
                n_workers=8, memory_limit='100GB')

client = Client(daskCluster)

client

In [ ]:
# with open('./out/{place}/what_cells_are_in_what_cluster_dict.json', 'r') as f:
#     includes_dict = json.load(f)
    
tessellation = gpd.read_parquet(f"./out/{place}/tessellation.pq")

buildings = gpd.read_parquet(f"./out/{place}/buildings.pq")

streets = gpd.read_parquet(f"./out/{place}/streets.pq").explode()

In [ ]:
tessellation.columns

In [ ]:
buildings.columns

In [ ]:
# # Map for comparison
# f, ax = plt.subplots(figsize=(200, 200))
# tessellation.plot(linewidth=0.8, ax=ax, edgecolor='0.8')
# streets.plot(ax=ax, color='blue')
# buildings.plot(ax=ax, color='black', alpha=0.5)
# ax.set_axis_off()

# # Show the plot
# plt.show()

In [ ]:
tessellation = tessellation[["uID", "geometry"]]

In [ ]:
buildings = buildings[["uID", "geometry"]]

In [ ]:
streets['nID'] = momepy.unique_id(streets)
streets = streets.reset_index().drop(["level_0", "level_1"], axis=1)
# buildings['nID'] = momepy.get_network_id(buildings, streets,
#                                                'nID').values

# buildings = buildings.drop_duplicates("uID").drop(columns="index_right")
# tessellation = tessellation.merge(buildings[['uID', 'nID']], on='uID', how='left')

In [ ]:
# @dask.delayed
# def queen_contiguity_load(n1, includes_dict):
#     chunk = gpd.read_parquet(f"./out/{place}/expanded_cells_in_chunk_{int(n1)}.pq")
    
#     # weights
    # queen_1 = libpysal.weights.w_subset(momepy.sw_high(k=1, gdf=chunk, ids='uID'), includes_dict[str(n1)]).to_adjlist()
    # queen_3 = libpysal.weights.w_subset(momepy.sw_high(k=3, gdf=chunk, ids='uID'), includes_dict[str(n1)]).to_adjlist()
    
#     return (queen_1, queen_3)

In [ ]:
# file_pattern = f"./out/{place}/expanded_cells_in_chunk_*.pq"
# file_list = glob.glob(file_pattern)
# num_files = len(file_list)

# weight_out = [process(n1, includes_dict) for n1 in tqdm(range(num_files))]

# weight_out = [queen_contiguity_load(n1, includes_dict) for n1 in range(num_files)]

# weight_out = dask.compute(weight_out)


In [ ]:
# queen_1 = libpysal.weights.W.from_adjlist(pd.concat(weight_out[0][i][0] for i in range(num_files)))

# queen_3 = libpysal.weights.W.from_adjlist(pd.concat(weight_out[0][i][1] for i in range(num_files)))

In [ ]:
# latlng_series = gpd.GeoSeries([Point(latlng)], crs=4326)

In [ ]:
# buildings["building_distance_to_central_city"] = buildings.geometry.centroid.distance(latlng_series.to_crs(local_crs)[0])

In [ ]:
# # Find the uIDs in tessellation that are not in buildings
# tessellation_uids = set(tessellation['uID'])
# buildings_uids = set(buildings['uID'])
# new_uids = tessellation_uids - buildings_uids

# # Create a new GeoDataFrame with the new uIDs and empty geometry
# new_rows = pd.DataFrame({'uID': list(new_uids)})
# new_rows['geometry'] = None
# new_gdf = gpd.GeoDataFrame(new_rows, crs=buildings.crs)

# # Append the new GeoDataFrame to buildings
# new_buildings = pd.concat([buildings, new_gdf], ignore_index=True)

In [ ]:
### Building Area
buildings['building_area'] = momepy.Area(buildings).series
tessellation['tess_area'] = momepy.Area(tessellation).series

In [ ]:
continuity = momepy.COINS(streets)
# stroke_gdf = continuity.stroke_gdf()

streets["continuity_stroke"] = continuity.stroke_attribute()

In [ ]:
queen_1 = libpysal.weights.fuzzy_contiguity(tessellation, tolerance=0.05, buffering=True, drop=True, buffer=5, ids="uID", silence_warnings=True)
queen_3 = momepy.sw_high(k=3, weights=queen_1)
dist200 = libpysal.weights.DistanceBand.from_dataframe(buildings, 200, ids='uID')

In [ ]:
buildings['building_circular_compactness'] = momepy.CircularCompactness(buildings).series
buildings['building_elongation'] = momepy.Elongation(buildings).series
buildings['building_squareness'] = momepy.Squareness(buildings).series
buildings['building_eri'] = momepy.EquivalentRectangularIndex(buildings).series
buildings['building_orientation'] = momepy.Orientation(buildings).series
streets['street_linearity'] = momepy.Linearity(streets).series
tessellation["tess_convexity"] = momepy.Convexity(tessellation).series
tessellation["tess_neighbours"] = momepy.Neighbors(tessellation, queen_3,'uID', weighted=True).series
tessellation["tess_neighbours_200"] =  momepy.Neighbors(tessellation, dist200, 'uID', weighted=True).series
tessellation["tess_covered_area"] = momepy.CoveredArea(tessellation, queen_1, "uID").series
tessellation["tess_area_iqr"] = momepy.Range(tessellation, values='tess_area', spatial_weights=queen_3, unique_id='uID', rng=(25, 75)).series
tessellation["tess_rea_theil"] = momepy.Theil(tessellation, values='tess_area', spatial_weights=queen_3, unique_id='uID').series
tessellation["tess_orientation"] = momepy.Orientation(tessellation).series
tessellation["building_neighbour_dist"] = momepy.NeighborDistance(buildings, queen_1, 'uID').series
tessellation["building_neighbourhood_interbuilding_distance"] = momepy.MeanInterbuildingDistance(buildings, queen_1, 'uID', 3).series

In [ ]:
# StreetProfile computation
profile = momepy.StreetProfile(streets, buildings)


In [ ]:
streets["street_width"] = profile.w
streets["street_width_deviation"] = profile.wd
streets["street_openness"] = profile.o

In [ ]:
buildings['building_weighted_circom'] = momepy.WeightedCharacter(
buildings, buildings["building_circular_compactness"], queen_3, 'uID', momepy.Area(buildings).series).series

In [ ]:
# primal = momepy.gdf_to_nx(streets, approach='primal')

In [ ]:
%%time
# momepy.node_degree(primal)

In [ ]:
%%time
# primal = momepy.closeness_centrality(primal, radius=400, name='closeness400', distance='mm_len', weight='mm_len')

In [ ]:
%%time
# primal = momepy.meshedness(primal, radius=400, distance="mm_len")

In [ ]:
# %%time
# primal = momepy.straightness_centrality(primal)

In [ ]:
# %%time
# primal = momepy.closeness_centrality(primal, name='closeness_global', weight='mm_len')

In [ ]:
# %%time
# primal =  momepy.betweenness_centrality(primal, name='betweenness_metric_n', mode='nodes', weight='mm_len', k=100)

In [ ]:
# %%time
# primal = momepy.straightness_centrality(primal)
# # momepy.mean_nodes(primal, 'straightness')
# momepy.mean_nodes(primal, 'closeness400')
# # momepy.mean_nodes(primal, 'closeness_global')
# # momepy.mean_nodes(primal, 'betweenness_metric_n')

# nodes, streets = momepy.nx_to_gdf(primal)

In [ ]:
cols = list(streets.columns)
cols.remove('geometry')

In [ ]:
streets_dropped = streets.drop(cols,axis=1).reset_index(drop=True)

In [ ]:
buildings = buildings.set_crs(local_crs)

In [ ]:
# buildings=buildings.drop(["index_right"], axis=1)

In [ ]:
buildings.columns

In [ ]:
buildings = gpd.sjoin_nearest(left_df=buildings, right_df=streets_dropped, how='left', max_distance=1000).drop_duplicates(subset=['geometry'])
duplicates = buildings[buildings.duplicated(['uID'], keep=False)]
if len(duplicates) > 0:
    print(f"There are {len(duplicates)} rows with duplicate uids.")
    print(duplicates)
else:
    print("There are no rows with duplicate uids.")


In [ ]:
buildings = buildings.dropna(subset=['geometry'])
tessellation = tessellation.dropna(subset=['geometry'])

In [ ]:
streets.reset_index(drop=True, inplace=True)

In [ ]:
streets["nID"]= streets.index

In [ ]:
buildings.drop(columns=['index_right'], inplace=True)

In [ ]:
streets["nID"]= streets.index
buildings =  gpd.sjoin_nearest(buildings, streets, how="left", max_distance=1000)

In [ ]:
buildings["building_str_align"] = momepy.StreetAlignment(buildings, streets,
                                   'building_orientation', 'nID',
                                   'nID').series

In [ ]:
buildings["building_str_align"]

In [ ]:
blg_cell_align = momepy.CellAlignment(buildings, tessellation,
                                      'building_orientation', 'tess_orientation',
                                      'uID', 'uID')
buildings['building_cell_align'] = blg_cell_align.series

In [ ]:
buildings['building_cell_align']

In [ ]:
tessellation['tess_car'] = momepy.AreaRatio(tessellation, buildings, 'tess_area', 'building_area', 'uID').series

In [ ]:
nodes = gpd.read_parquet(f"./out/{place}/vertices_frame.pq")
edges = gpd.read_parquet(f"./out/{place}/edges_frame.pq")
dual = gpd.read_parquet(f"./out/{place}/dual_vertices_frame.pq")

In [ ]:
buildings = buildings.drop(["nID", "index_right", "nID"], axis=1)
nodes = nodes.drop(["vertID"], axis=1)
edges = edges.drop("edgeID", axis=1)

In [ ]:
buildings = gpd.sjoin_nearest(left_df=buildings, right_df=nodes, how='left', max_distance=1000).drop_duplicates(subset=['geometry'])
duplicates = buildings[buildings.duplicated(['uID'], keep=False)]
if len(duplicates) > 0:
    print(f"There are {len(duplicates)} rows with duplicate uids.")
    print(duplicates)
else:
    print("There are no rows with duplicate uids.")


In [ ]:
buildings = buildings.drop("index_right", axis=1)
buildings = gpd.sjoin_nearest(left_df=buildings, right_df=edges, how='left', max_distance=1000).drop_duplicates(subset=['geometry'])
duplicates = buildings[buildings.duplicated(['uID'], keep=False)]
if len(duplicates) > 0:
    print(f"There are {len(duplicates)} rows with duplicate uids.")
    print(duplicates)
else:
    print("There are no rows with duplicate uids.")


In [ ]:
buildings = buildings.drop("index_right", axis=1)
buildings = gpd.sjoin_nearest(left_df=buildings, right_df=dual, how='left', max_distance=1000).drop_duplicates(subset=['geometry'])
duplicates = buildings[buildings.duplicated(['uID'], keep=False)]
if len(duplicates) > 0:
    print(f"There are {len(duplicates)} rows with duplicate uids.")
    print(duplicates)
else:
    print("There are no rows with duplicate uids.")


In [ ]:
tessellation = tessellation.merge(buildings, how='left', left_on = "uID", right_on="uID" )

In [ ]:
tessellation = tessellation.drop(["geometry_y", "index_right"], axis=1)
tessellation = tessellation.rename(columns={"geometry_x": "geometry"})
tessellation = gpd.GeoDataFrame(tessellation.dropna(subset=['geometry']))

In [ ]:
with open(f"./out/{place}/weights.pickle", 'wb') as f:
    pickle.dump(queen_1, f)
    pickle.dump(queen_3, f)
    pickle.dump(dist200, f)
    f.close()

In [ ]:
tessellation.columns=tessellation.columns.astype(str)

In [ ]:
tessellation

In [ ]:
tessellation.describe()

In [ ]:
tessellation.to_parquet(f"./out/{place}/tessellation_stats_no_percentile.pq")

In [ ]:
# def split_dataframe(df, num_dfs):
#     """
#     This function splits a DataFrame into a list of smaller DataFrames with equal number of columns.

#     Parameters:
#     df (pandas.DataFrame): The DataFrame to split.
#     num_dfs (int): The number of DataFrames to split 'df' into.

#     Returns:
#     list: A list of pandas DataFrame objects.
#     """

#     dfs = []
#     n = df.shape[1]
#     num_cols = n // num_dfs

#     for i in range(num_dfs):
#         start = i * num_cols
#         end = (i + 1) * num_cols if i != num_dfs - 1 else n
#         dfs.append(df.iloc[:, start:end])

#     return dfs

# # Use the function
# list_of_dfs = split_dataframe(tessellation[tessellation.columns.drop(["uID", "geometry"])], 16)

In [ ]:
dropped = tessellation.drop(["uID", "geometry"], axis=1)

column_lists = [list(dropped.columns[i:i + 16]) 
                for i in range(0, len(dropped.columns), 16)]



In [ ]:
# def process_percentiles(tessellation, queen_3):
#     percentiles = []
#     for column in tessellation.columns.drop(["uID", "geometry"]):
#         perc = momepy.Percentiles(tessellation, column, queen_3, "uID", verbose=False).frame
#         perc.columns = [f"{column}_" + str(x) for x in perc.columns]
#         percentiles.append(perc)
        
#     percentiles_joined = pd.concat(percentiles, axis=1)
#     return percentiles_joined

In [ ]:
def process_percentiles(df, column, queen_3):
    perc = momepy.Percentiles(df, column, queen_3, "uID", verbose=False).frame
    perc.columns = [f"{column}_" + str(x) for x in perc.columns]
    
    return perc

In [ ]:
tessellation.columns

In [ ]:
queen_3_future = client.scatter(queen_3)

In [ ]:
listofdf = []
for column_list in column_lists:
    computations = []

    for column in column_list:
        computations.append(delayed(process_percentiles)(tessellation[['uID', column]], column, queen_3_future))
        
    out = client.compute(computations)
    
    
    {listofdf.append(out[i].result()) for i in range(len(column_list))}

In [ ]:
percentiles_joined = pd.concat(listofdf, axis=1)

In [ ]:
merged_df = pd.concat([percentiles_joined, tessellation[['uID', 'geometry']]], axis=1)

In [ ]:
percentiles_joined["uID"] = tessellation["uID"]
percentiles_joined["geometry"] = tessellation["geometry"]

In [ ]:
gdf = gpd.GeoDataFrame(percentiles_joined, geometry="geometry")

In [ ]:
gdf.plot()

In [ ]:
gdf.to_parquet(f"./out/{place}/tessellation_stats.pq")

In [ ]:
# chunk_size = 250000
# chunks = []
# for i in range(0, len(tessellation), chunk_size):
#     chunks.append(tessellation.iloc[i:i + chunk_size])

In [ ]:
# import os
# new_directory = f"./out/{place}/tessellation/"

# if not os.path.exists(new_directory):
#     os.makedirs(new_directory)
#     print(f"Created directory: {new_directory}")
# else:
#     print(f"Directory already exists: {new_directory}")

# # iterate over all files in the directory and delete them
# for filename in os.listdir(new_directory):
#     file_path = os.path.join(new_directory, filename)
#     if os.path.isfile(file_path):
#         os.remove(file_path)
#         print(f"Deleted file: {file_path}")

In [ ]:
# for i, chunk in enumerate(chunks):
#     chunk.to_parquet(f"./out/{place}/tessellation/tesselation_chunk_{i}.pq")

In [ ]:
# duplicates = tessellation[tessellation.duplicated(['uID'], keep=False)]
# if len(duplicates) > 0:
#     print(f"There are {len(duplicates)} rows with duplicate uids.")
#     print(duplicates)
# else:
#     print("There are no rows with duplicate uids.")

In [ ]:
# import pickle

# # Open the file for reading
# with open('./out/{place}/weights.pickle', 'rb') as f:
#     # Load the variables in the same order that they were dumped
#     queen_1 = pickle.load(f)
#     queen_3 = pickle.load(f)
#     dist200 = pickle.load(f)
#     tessellation = pickle.load(f)

In [ ]:
# tessellation.to_parquet("./out/{place}/tessellation_with_data.pq", write_metadata_file=False)

In [ ]:
daskCluster.close()
client.shutdown()
